# Introduzione allo sviluppo di interpreti

## TODO:
Esempi di risolutore espressioni e interprete lambda calcolo

Risolutore espressioni:

- big step semantics (eval) e small step (eval_step)
- parser discendente ricorsivo (rivedere usando funzione lookahead)
- aggiungere variabili con ambiente e sostituzione?

Lambda calcolo:

- da Giangi

Esempio: le espressioni aritmetiche (su interi)

```
Exp ::= n | Exp op Exp | -Exp | (Exp)
op ::= + | - | * | / | %
```

In [1]:
type op = Add | Sub | Mul | Div | Mod ;;
type exp =
    | Val of int
    | Op of op*exp*exp
    | Neg of exp ;;

type op = Add | Sub | Mul | Div | Mod


type exp = Val of int | Op of op * exp * exp | Neg of exp


In [2]:
let exp1 = Op (Sub, (Op (Mul, Val 3, Val 7)), Val 5) ;;
let exp2 = Op (Mul, Val 3, (Op (Sub, Val 7, Val 5))) ;;

val exp1 : exp = Op (Sub, Op (Mul, Val 3, Val 7), Val 5)


val exp2 : exp = Op (Mul, Val 3, Op (Sub, Val 7, Val 5))


In [3]:
let symbol o =
    match o with
    | Add -> "+" | Sub -> "-" | Mul -> "*" | Div -> "/" | Mod -> "%" ;;
let rec to_string e =
    match e with
    | Val n -> string_of_int n
    | Op (o,e1,e2) -> "("^ (to_string e1) ^ (symbol o) ^ (to_string e2) ^")"
    | Neg e' -> "-"^(to_string e') ;;

val symbol : op -> string = <fun>


val to_string : exp -> string = <fun>


In [4]:
let rec eval e =
    match e with
    | Val n -> n
    | Op (o,e1,e2) -> 
        (match o with
        | Add -> (eval e1) + (eval e2)
        | Sub -> (eval e1) - (eval e2)
        | Mul -> (eval e1) * (eval e2)
        | Div -> (eval e1) / (eval e2)
        | Mod -> (eval e1) mod (eval e2)
        )
    | Neg e' -> - (eval e') ;;

val eval : exp -> int = <fun>


In [5]:
let rec eval_step e =
        match e with
        | Val n -> Val n
        | Op (o,e1,e2) ->
            (match e1,e2 with
            | Val _,Val _ -> Val (eval e)
            | Op (_,_,_),_ | Neg _,_ -> Op (o,(eval_step e1),e2)
            | _,Op (_,_,_) | _,Neg _ -> Op (o,e1,(eval_step e2))
            )
        | Neg e' ->
            (match e' with
            | Val n' -> Val (-n')
            | _ -> Neg (eval_step e')
            )
;;

exp1;;
eval_step exp1;;
eval_step (eval_step exp1);;
eval exp1;;

val eval_step : exp -> exp = <fun>


- : exp = Op (Sub, Op (Mul, Val 3, Val 7), Val 5)


- : exp = Op (Sub, Val 21, Val 5)


- : exp = Val 16


- : int = 16


In [6]:
let rec solve e =
    match e with
    | Val n -> (to_string e)
    | _ -> (to_string e)^" = "^(solve (eval_step e)) ;;

solve exp1 ;;

val solve : exp -> string = <fun>


- : string = "((3*7)-5) = (21-5) = 16"


```
Exp ::= ( Exp ) Exp' | n Exp' | - Exp
Exp'::= op Exp Exp' |
```

In [7]:
 let rec tokenize s t=
    if String.length s = 0 then t
    else
        let is_symbol c =
            match c with
            | "(" | ")" | "+" | "-" | "*" | "/" | "%" -> true
            | _ -> false
        in
        let is_digit c =
            match c with
            | "0" | "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9" -> true
            | _ -> false
        in 
            let c = String.sub s 0 1
        in
            let s' = String.sub s 1 ((String.length s) -1)
        in 
            if c=" " then tokenize s' t
            else if is_symbol c then c::(tokenize s' t)
            else if is_digit c then
                match tokenize s' t with
                | [] -> [c]
                | c'::t' -> if is_symbol c' then c::(tokenize s' t)
                            else (c^c')::(tokenize s' t')
            else ["PARSE ERROR"] (* errore: caratteri o altri simboli non ammessi *)
    ;;    

let t = tokenize "(3 + 4) - (2 * 2)" [];;

val tokenize : string -> string list -> string list = <fun>


val t : string list = ["("; "3"; "+"; "4"; ")"; "-"; "("; "2"; "*"; "2"; ")"]


In [8]:
type n_exp = 
    | Par of n_exp*n_exp'
    | Num of int*n_exp'
    | Minus of n_exp
    | Error1
and n_exp' =
    | Oper of op*n_exp*n_exp' 
    | Empty 
    | Error2 ;;

let parse_tokens t =
    let rec exp t =
        match t with
        | [] | "PARSE_ERROR"::_ | "+"::_ | "*"::_ | "/"::_ | "%"::_ | ")"::_ -> (Error1,t)
        | "("::t' -> let (e,t'') = exp t' in 
                        (match t'' with 
                        | ")"::t''' -> let (e',t'''') = exp' t''' in (Par (e,e'),t'''')
                        | _ -> (Error1,t'')
                        )
        | "-"::t' -> let (e,t'') = (exp t') in (Minus e,t'')
        | s::t' -> let (e,t'') = (exp' t') in (Num ((int_of_string s),e),t'') (* s dovrebbe essere per certo un numero... *)
    and exp' t =
        match t with
        | [] -> (Empty,[])
        | ")"::t' -> (Empty,t) (* caso epsilon *)
        | "+"::t' -> let (e,t'') = (exp t')
                     in (let (e',t''') = (exp' t'')
                         in (Oper (Add,e,e'),t'''))
        | "-"::t' -> let (e,t'') = (exp t')
                     in (let (e',t''') = (exp' t'')
                         in (Oper (Sub,e,e'),t'''))
        | "*"::t' -> let (e,t'') = (exp t')
                     in (let (e',t''') = (exp' t'')
                         in (Oper (Mul,e,e'),t'''))
        | "/"::t' -> let (e,t'') = (exp t')
                     in (let (e',t''') = (exp' t'')
                         in (Oper (Div,e,e'),t'''))
        | "%"::t' -> let (e,t'') = (exp t')
                     in (let (e',t''') = (exp' t'')
                         in (Oper (Mod,e,e'),t'''))
        | _ -> (Error2,[])
    in
        exp t;;


(*l*)

let t = tokenize "(6+(2)*3)+(3/4)" [];;
let e = parse_tokens t;;

type n_exp =
    Par of n_exp * n_exp'
  | Num of int * n_exp'
  | Minus of n_exp
  | Error1
and n_exp' = Oper of op * n_exp * n_exp' | Empty | Error2


val parse_tokens : string list -> n_exp * string list = <fun>


val t : string list =
  ["("; "6"; "+"; "("; "2"; ")"; "*"; "3"; ")"; "+"; "("; "3"; "/"; "4"; ")"]


val e : n_exp * string list =
  (Par
    (Num (6,
      Oper (Add, Par (Num (2, Empty), Oper (Mul, Num (3, Empty), Empty)),
       Empty)),
    Oper (Add, Par (Num (3, Oper (Div, Num (4, Empty), Empty)), Empty),
     Empty)),
   [])


In [9]:
let rec convert ne =
    match ne with
    | Par (e,e') -> 
        (match e' with
        | Oper (o,e1,e1') -> Op (o,convert e,convert (Par (e1,e1')))
        | Empty -> convert e
        | Error2 -> Val (-9999) (* in caso di errore... *)
        )
    | Num (n,e') ->
        (match e' with
        | Oper (o,e1,e1') -> Op (o,Val n,convert (Par (e1,e1')))
        | Empty -> Val n
        | Error2 -> Val (-9999) (* in caso di errore... *)
        )
    | Minus e -> Neg (convert e)
    | Error1 -> Val (-9999)  (* in caso di errore... *)
;;
            
let (e',l) = e ;;
let e2 = convert e';;
eval e2 ;;
solve e2 ;;

eval ( convert ( fst (parse_tokens (tokenize "(-5*2)/(1+1)" []))));;
solve ( convert ( fst (parse_tokens (tokenize "(-5*2)/(1+1)" []))));;

val convert : n_exp -> exp = <fun>


val e' : n_exp =
  Par
   (Num (6,
     Oper (Add, Par (Num (2, Empty), Oper (Mul, Num (3, Empty), Empty)),
      Empty)),
   Oper (Add, Par (Num (3, Oper (Div, Num (4, Empty), Empty)), Empty), Empty))
val l : string list = []


val e2 : exp =
  Op (Add, Op (Add, Val 6, Op (Mul, Val 2, Val 3)), Op (Div, Val 3, Val 4))


- : int = 12


- : string = "((6+(2*3))+(3/4)) = ((6+6)+(3/4)) = (12+(3/4)) = (12+0) = 12"


- : int = -5


- : string = "(-(5*2)/(1+1)) = (-10/(1+1)) = (-10/(1+1)) = (-10/2) = -5"
